In [ ]:

#Basic py: 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import math
import json
from urllib.request import urlopen
import glob
#Vis
#import seaborn as sns

#Geo
import geopandas as gpd
import fiona
from shapely.geometry import Point
import descartes
import contextily as ctx #Basemaps 
import xarray as xr 

# Preperation to get TRI data into STILT ready format: 


In [97]:
#Load TRI dat: 
tri_df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/TRI_base_process_90_99.csv').drop(columns=['Unnamed: 0'])

In [98]:
#This seperates fugitive and stack releases - setting the stack height of the release for fugitive releases to 0
fug = tri_df[tri_df['51-FUGITIVEAIR']>0]
fug['StackHeight']=0
fug = fug.rename(columns = {'51-FUGITIVEAIR':'Release (lbs/year)'})
fug = fug.drop(columns = ['52-STACKAIR'])

stack = tri_df[tri_df['52-STACKAIR']>0]
stack = stack.rename(columns = {'52-STACKAIR':'Release (lbs/year)'})
stack = stack.drop(columns = ['51-FUGITIVEAIR'])

stack_fug_df = pd.concat([stack,fug])

In [100]:
stack_fug_df['StackHeight']

0      30.5
1      10.7
2      10.7
3      10.7
4      10.7
       ... 
895     0.0
896     0.0
897     0.0
898     0.0
899     0.0
Name: StackHeight, Length: 1379, dtype: float64

In [111]:
def STILT_converter(df,min_year,max_year,save_base_name):
    """
    Converts a dataframe containing LATITUDE LONGITUDE Stackheight, CHEMICAL and a zagl to a csv for STILT pipeline. 

    ===
    Inputs:
    1. df - dataframe containing expanded fugitive and stack releases, renamed zagl. 
    2. min_year - minimum year of TRI releases 
    3. max_year - maximum year of analysis for TRI releases
    4. save_base_name - the base save name for an id mappings file (USE FOR JOIN after STILT RUN) and a stilt run (USE FOR STILT) csv file

    Outputs: 
    1. Saves all files - no output returned 
    """

    #Create a base dataframe which houses all stilt runs as seperated by lat/long/stackheight/chemical/amount and year. 
    base_df = df[(df.YEAR >= min_year) & (df.YEAR <=max_year)][['LATITUDE','LONGITUDE','StackHeight','CHEMICAL','Release (lbs/year)','YEAR']].rename(columns={'LATITUDE':'lati','LONGITUDE':'long','CHEMICAL':'Chemical','StackHeight':'zagl'})

    #Stilt only runs particles based upon location and time (concentration unnessecary. Create a subset to run simulations on and an id to remerge on (for convolutional toxicity calculation))
    stilt_run_id = base_df.drop_duplicates(['lati','long','zagl','YEAR']).drop(columns=['Chemical','Release (lbs/year)']).sort_values(by='YEAR').reset_index(drop=True).reset_index().rename(columns = {'index':'id'})

    #Add the id to the base_df 
    stilt_trace_mapping = base_df.merge(stilt_run_id, on=['lati','long','zagl','YEAR']).sort_values(by='id')

    #save the files
    stilt_trace_mapping.to_csv(str(save_base_name + '_id_mappings.csv'),index=False)
    stilt_run_id.to_csv(str(save_base_name + '_stilt_RUN.csv'),index = False)


In [112]:
STILT_converter(stack_fug_df,1990,1999,'/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/unique_TRI_location_height_year')